In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

from scipy.stats import spearmanr

In [ ]:
def pearson_r_and_err(x, y, bootstrap_repeats=1000):
    assert x.ndim == y.ndim == 1, "x and y must be 1D array"
    assert x.shape[0] == y.shape[0], "x and y does not have the same len" 
    
    pearson_r = np.corrcoef(x, y)[0, -1]
    nsamples = x.shape[0]
    
    rs = []
    for _ in range(bootstrap_repeats):
        rand_idx = np.random.choice(nsamples, size=nsamples, replace=True)
        r = np.corrcoef(x[rand_idx], y[rand_idx])[0, -1]
        if (not np.isnan(r)) and (r not in [np.inf, -np.inf]):
            rs.append(r)

    rs = np.array(rs)
    err = rs.std()
    return pearson_r, err


def rmse_and_err(x, y, bootstrap_repeats=1000):
    assert x.ndim == y.ndim == 1, "x and y must be 1D array"
    assert x.shape[0] == y.shape[0], "x and y does not have the same len" 
    
    all_sample_rmse = np.sqrt(mean_squared_error(x, y))
    nsamples = x.shape[0]
    
    rmses_boostrap = []
    for _ in range(bootstrap_repeats):
        rand_idx = np.random.choice(nsamples, size=nsamples, replace=True)
        rmse = np.sqrt(mean_squared_error(x[rand_idx], y[rand_idx]))
        
        if (not np.isnan(rmse)) and (rmse not in [np.inf, -np.inf]):
            rmses_boostrap.append(rmse)

    rmses_boostrap = np.array(rmses_boostrap)
    err = rmses_boostrap.std()
    return all_sample_rmse, err


def spearman_r_and_err(x, y, bootstrap_repeats=1000):
    assert x.ndim == y.ndim == 1, "x and y must be 1D array"
    assert x.shape[0] == y.shape[0], "x and y does not have the same len" 
    
    spearman_r, _ = spearmanr(x, y)
    nsamples = x.shape[0]
    
    rs = []
    for _ in range(bootstrap_repeats):
        rand_idx = np.random.choice(nsamples, size=nsamples, replace=True)
        r, _ = spearmanr(x[rand_idx], y[rand_idx])
        if (not np.isnan(r)) and (r not in [np.inf, -np.inf]):
            rs.append(r)

    rs = np.array(rs)
    err = rs.std()
    return spearman_r, err

# Basic metrics

## Linear regression

In [ ]:
df = pd.read_csv("results/lr/test_pred.csv")
print(df.shape)
df.head()

In [ ]:
rmse, err = rmse_and_err(df["dG"], df["pred"])
print("RMSE = %0.3f +/- %0.3f" %(rmse, err))

pearson_r, err = pearson_r_and_err(df["dG"], df["pred"])
print("Pearson's R = %0.3f +/- %0.3f" %(pearson_r, err))

spearman_r, err = spearman_r_and_err(df["dG"], df["pred"])
print("Spearman's Rho = %0.3f +/- %0.3f" %(spearman_r, err))

## Random forest

In [ ]:
df = pd.read_csv("results/rf/test_pred.csv")
print(df.shape)
df.head()

In [ ]:
rmse, err = rmse_and_err(df["dG"], df["pred"])
print("RMSE = %0.3f +/- %0.3f" %(rmse, err))

pearson_r, err = pearson_r_and_err(df["dG"], df["pred"])
print("Pearson's R = %0.3f +/- %0.3f" %(pearson_r, err))

spearman_r, err = spearman_r_and_err(df["dG"], df["pred"])
print("Spearman's Rho = %0.3f +/- %0.3f" %(spearman_r, err))

## XGBoost

In [ ]:
df = pd.read_csv("results/xgb/test_pred.csv")
print(df.shape)
df.head()

In [ ]:
df["pred"].sort_values()

In [ ]:
rmse, err = rmse_and_err(df["dG"], df["pred"])
print("RMSE = %0.3f +/- %0.3f" %(rmse, err))

pearson_r, err = pearson_r_and_err(df["dG"], df["pred"])
print("Pearson's R = %0.3f +/- %0.3f" %(pearson_r, err))

spearman_r, err = spearman_r_and_err(df["dG"], df["pred"])
print("Spearman's Rho = %0.3f +/- %0.3f" %(spearman_r, err))

## Graph-Conv

In [ ]:
df = pd.read_csv("results/graph_conv/test_pred.csv")
print(df.shape)
df.head()

In [ ]:
# Experiment vs pred test scatter plot

In [ ]:
def _regression_line(x, y):
    coeffs = np.polyfit(x, y, 1)
    intercept = coeffs[-1]
    slope = coeffs[-2]
    min_x = np.min(x)
    max_x = np.max(x)
    xl = np.array([min_x, max_x])
    yl = slope*xl + intercept
    line_eq_text = "y = %0.2fx"%slope
    if intercept >= 0:
        line_eq_text += " + "
    else:
        line_eq_text += " - "
    line_eq_text += "%0.2f"%np.abs(intercept)
    return xl, yl, line_eq_text


def _correlation_coef(x, y, repeats=100):
    x = np.array(x)
    y = np.array(y)

    corr_coef = np.corrcoef([x, y] )[0][-1]

    # estimate std
    rs = []
    for repeat in range(repeats):
        sel_ind = np.random.choice(len(x), size=len(x), replace=True)
        sel_x = x[sel_ind]
        sel_y = y[sel_ind]
        rs.append( np.corrcoef([sel_x, sel_y] )[0][-1] )
    std = np.std(rs)
    return corr_coef, std


def _rmse(x,y, repeats=100):
    x = np.array(x)
    y = np.array(y)

    mean_sq_error = (x - y)**2
    mean_sq_error = np.sqrt(mean_sq_error.mean())

    # estimate std
    rmses = []
    for repeat in range(repeats):
        sel_ind = np.random.choice(len(x), size=len(x), replace=True)
        sel_x = x[sel_ind]
        sel_y = y[sel_ind]
        rmse = (sel_x - sel_y)**2 
        rmse = np.sqrt(rmse.mean())
        rmses.append(rmse)

    std = np.std(rmses)
    return mean_sq_error, std


def _armse(x,y, repeats=100):
    x = np.array(x)
    y = np.array(y)
    mean_sq_error = (x - y - (x.mean() - y.mean()) )**2
    mean_sq_error = np.sqrt(mean_sq_error.mean())

    # estimate std
    rmses = []
    for repeat in range(repeats):
        sel_ind = np.random.choice(len(x), size=len(x), replace=True)
        sel_x = x[sel_ind]
        sel_y = y[sel_ind]
        rmse = ( sel_x - sel_y - (sel_x.mean() - sel_y.mean()) )**2
        rmse = np.sqrt(rmse.mean())
        rmses.append(rmse)

    std = np.std(rmses)
    return mean_sq_error, std


def _scatter_plot_info(x, y, label):
    xl, yl, line_eq_text = _regression_line(x, y)
    corr_coef, r_std = _correlation_coef(x, y)
    rmse, rmse_std = _rmse(x,y)
    armse, armse_std = _armse(x,y)

    text = "\n"+label
    text += "\nPearson's R = %0.2f (%0.2f)"%(corr_coef, r_std)
    text += "\nRMSE = %0.2f (%0.2f)"%(rmse, rmse_std)
    text += "\naRMSE = %0.2f (%0.2f)"%(armse, armse_std)
    text += "\n%s\n"%line_eq_text
    return text


def scatter_plot_info(x, y, ligands, out, all_only=False, stdout=False):
    text = _scatter_plot_info(x, y, "All")
    if all_only:
        open(out, "w").write(text)
        return None

    x_ac, y_ac = [], []
    x_inac, y_inac = [], []
    for i, ligand in enumerate(ligands):
        if ".inactive." in ligand or ligand == "phenol.A__AAA":
            x_inac.append(x[i])
            y_inac.append(y[i])
        else:
            x_ac.append(x[i])
            y_ac.append(y[i])

    if len(x_ac) > 0:
        text += _scatter_plot_info(x_ac, y_ac, "active")
    if len(x_inac) > 0:
        text += _scatter_plot_info(x_inac, y_inac, "inactive")

    open(out, "w").write(text)
    if stdout:
        print(text)
    return None


def scatter_plot(x, y, xlabel, ylabel, out, 
                show_xy_axes=True,
                xerr=None, yerr=None,
                xlimits=None,
                ylimits=None,
                aspect="auto",

                same_xy_scale=True,
                show_regression_line=False,
                show_diagonal_line=False,

                show_rmse=False,
                show_armse=False,
                show_R=False,
                show_regression_line_eq=False,

                figure_size=(3.2, 3.2*6/8),
                dpi=300, 
                fontsize=8,
                font = {"fontname": "Arial"},
                markers=None,
                markersize=20, 
                markercolors=None,
                text_pos=[0.55, 0.1],
                line_styles = {"regression" : "k--", "diagonal" : "k-"},
                line_weights = {"regression" : 2, "diagonal" : 1} ):
    """
    """
    if show_diagonal_line:
        assert same_xy_scale, "to show diagonal line, x and y must be shown on the same scale"

    assert x.shape == y.shape, "x and y must have the same shape"
    if xerr is not None:
        assert xerr.shape == x.shape, "x and xerr must have the same shape"
        assert np.all(xerr >= 0), "xerr must be non-negative"

    if yerr is not None:
        assert yerr.shape == y.shape, "y and yerr must have the same shape"
        assert np.all(yerr >= 0), "yerr must be non-negative"

    if xerr is None:
        lower_x, upper_x = x.min(), x.max()
    else:
        lower_x = (x - xerr).min()
        upper_x = (x + xerr).max()

    if yerr is None:
        lower_y, upper_y = y.min(), y.max()
    else:
        lower_y = (y - yerr).min()
        upper_y = (y + yerr).max()

    if same_xy_scale:
        lower_x = np.min([lower_x, lower_y])
        lower_y = lower_x

        upper_x = np.max([upper_x, upper_y])
        upper_y = upper_x

    lower_x = np.floor(lower_x)
    upper_x = np.ceil(upper_x)
    lower_y = np.floor(lower_y)
    upper_y = np.ceil(upper_y)

    if xlimits is not None:
        lower_x, upper_x = xlimits

    if ylimits is not None:
        lower_y, upper_y = ylimits

    plt.figure(figsize=figure_size)
    if show_diagonal_line:
        plt.plot( [lower_x, upper_x], [lower_y, upper_y], line_styles["diagonal"], line_weights["diagonal"] )

    plt.axis(aspect=aspect)

    axes = plt.gca()
    axes.set_xlim([lower_x, upper_x])
    axes.set_ylim([lower_y, upper_y])


    if markercolors is None:
        markercolors = ["k" for i in range(len(x))]
    else:
        assert type(markercolors) == list, "markercolors must be either None or a list of str"
        assert len(markercolors) == len(x), "markercolors must have the same len as x and y"

    if markers is None:
        markers = ["o" for i in range(len(x))]
    else:
        assert type(markers) == list, "markers must be either None or a list of str"
        assert len(markers) == len(x), "markers must have the same len as x and y"

    if xerr is None and yerr is None:
        for i in range(len(x)):
            plt.errorbar(x[i], y[i], ms=markersize, marker=markers[i], c=markercolors[i], linestyle="None")

    if (xerr is not None) and (yerr is None):
        for i in range(len(x)):
            plt.errorbar(x[i], y[i], xerr=xerr[i], ms=markersize, marker=markers[i], c=markercolors[i], linestyle="None")

    if (xerr is None) and (yerr is not None):
        for i in range(len(x)):
            plt.errorbar(x[i], y[i], yerr=yerr[i], ms=markersize, marker=markers[i], c=markercolors[i], linestyle="None")

    if (xerr is not None) and (yerr is not None):
        for i in range(len(x)):
            plt.errorbar(x[i], y[i], xerr=xerr[i], yerr=yerr[i], ms=markersize, marker=markers[i], c=markercolors[i], linestyle="None")

    if show_regression_line:
        xl, yl, line_text = _regression_line(x, y)
        plt.plot(xl, yl, line_styles["regression"], lw=line_weights["regression"])

    text = ""
    if show_R:
        corr_coef, r_std = _correlation_coef(x, y)
        text += "Pearson's R = %0.2f"%(corr_coef)

    if show_rmse:
        mean_sq_error, rmse_std = _rmse(x,y)
        text += "\nRMSE = %0.2f"%(mean_sq_error)

    if show_armse:
        amean_sq_error, armse_std = _armse(x,y)
        text += "\naRMSE = %0.2f"%(amean_sq_error)

    if show_regression_line_eq:
        xl, yl, line_text = _regression_line(x, y)
        text += "\n%s"%line_text
    if len(text) > 0:
        plt.text( text_pos[0]*upper_x + ( 1-text_pos[0] )*lower_x, text_pos[1]*upper_y + ( 1-text_pos[1] )*lower_y, text, fontsize=fontsize, **font )

    ax = plt.axes()
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(fontsize)

    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(fontsize)

    if not show_xy_axes:
        x_label = None
        y_label = None
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    if xlabel is not None:
        plt.xlabel(xlabel, fontsize=fontsize, **font)
    if ylabel is not None:
        plt.ylabel(ylabel, fontsize=fontsize, **font)

    plt.tight_layout()
    plt.savefig(out, dpi=dpi)
    return None


## xgb

In [ ]:
test_pred_df = pd.read_csv("results/xgb/test_pred.csv")
out = "figures/experiment_vs_pred_scatter_plot_for_test_xgb.pdf"

x = test_pred_df["dG"]
y = test_pred_df["pred"]

xlabel = "Experimental $\Delta G$ (kcal/mol)"
ylabel = "Predicted $\Delta G$ (kcal/mol)"


scatter_plot(x, y, xlabel, ylabel, out, 
                show_xy_axes=True,
                xerr=None, yerr=None,
                xlimits=None,
                ylimits=None,
                aspect="auto",

                same_xy_scale=True,
                show_regression_line=True,
                show_diagonal_line=False,

                show_rmse=False,
                show_armse=False,
                show_R=False,
                show_regression_line_eq=False,

                figure_size=(3.2, 3.2*6/8),
                dpi=300, 
                fontsize=8,
                font = {"fontname": "Arial"},
                markers=None,
                markersize=4, 
                markercolors=None,
                text_pos=[0.55, 0.1],
                line_styles = {"regression" : "k--", "diagonal" : "k-"},
                line_weights = {"regression" : 2, "diagonal" : 1} )

# Energy distribution

In [ ]:
def get_bin_edges_labels(min_val, max_val, nbins):
    bin_edges = np.linspace(min_val, max_val, nbins+1)
    bin_labels = (bin_edges[:-1] + bin_edges[1:]) / 2
    return bin_edges, bin_labels


def bin_count(x, bin_edges, bin_labels):
    x_bins = pd.cut(x, bins=bin_edges, labels=bin_labels)
    hist = x_bins.value_counts()
    hist = hist / hist.sum()
    hist = hist.sort_index()
    bin_center = hist.index.to_numpy()
    hist_pct = hist.values
    return bin_center, hist_pct

In [ ]:
dg_train = pd.read_csv("data/process/pdY_train.csv")["dG"]
dg_test = pd.read_csv("data/process/pdY_test.csv")["dG"]

dg_test_pred = pd.read_csv("results/xgb/test_pred.csv")["pred"]    
dg_vietherbs_pred = pd.read_csv("results/xgb/vietherbs.csv")["pred"] 
dg_chembl_27_pred = pd.read_csv("results/xgb/chembl_27.csv")["pred"]


MIN_VAL = -15
MAX_VAL = 0
nbins = 14

bin_edges, bin_labels = get_bin_edges_labels(MIN_VAL, MAX_VAL, nbins)

hist_train = bin_count(dg_train, bin_edges, bin_labels)
hist_test = bin_count(dg_test, bin_edges, bin_labels)

hist_test_pred = bin_count(dg_test_pred, bin_edges, bin_labels)
hist_vietherbs = bin_count(dg_vietherbs_pred, bin_edges, bin_labels)
hist_chembl_27 = bin_count(dg_chembl_27_pred, bin_edges, bin_labels)


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))
plt.subplots_adjust(wspace=0.2)


axes[0].plot(hist_train[0], hist_train[1], label="train", c="b")
axes[0].plot(hist_test[0], hist_test[1], label="test", c="r")
axes[0].legend(fontsize=10)
axes[0].set_ylabel("Density")
axes[0].set_xlabel("Experimental $\Delta G$")

axes[1].plot(hist_test_pred[0], hist_test_pred[1], label="test", c="r")
#axes[1].plot(hist_vietherbs[0], hist_vietherbs[1], label="Vietherbs", c="g")
axes[1].plot(hist_chembl_27[0], hist_chembl_27[1], label="Chembl 27", c="g")
axes[1].legend(fontsize=10)
axes[1].set_xlabel("Predicted $\Delta G$")

plt.tight_layout()
out = "figures/dg_distribution_xgb_Chembl.pdf"
plt.savefig(out, dpi=300)

In [ ]:
dg_train.min(), dg_test.min(), dg_test_pred.min(), dg_chembl_27_pred.min()

In [ ]:
dg_train.max(), dg_test.max(), dg_test_pred.max(), dg_chembl_27_pred.max()

In [ ]:
MIN_VAL = -13
MAX_VAL = -5
nbins = 16

bin_edges, bin_labels = get_bin_edges_labels(MIN_VAL, MAX_VAL, nbins)

In [ ]:
dg_test_pred